<a href="https://colab.research.google.com/github/olakoja/DA-MUS/blob/main/%D0%91%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82_%D0%B4%D0%BB%D1%8F_%D0%BF%D0%BE%D0%B4%D0%B3%D0%BE%D1%82%D0%BE%D0%B2%D0%BA%D0%B8_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%BF%D0%B5%D1%80%D0%B5%D0%B4_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Зачем каждая часть

Понадобятся данные:

*   о предполагаемых категориях пространств. Вот ссылка на код, который поможет их получить: https://colab.research.google.com/drive/1q4j3dcMevuS0ZZ0uJa8Rf6-X4WgXTnqs#scrollTo=w0RzQ8OoFR10
*   о плотности точек внутри полигонов;
*   о расстоянии от центроида до ближайшей точки и
*   о линейных объектах, пересекающих полигоны
*   о количестве точек на расстоянии от полигона (например, сколько продуктовых магазинов на расстоянии 100 м. от парка)
Если необходимо сделать "нарезку" полигональных объектов линейными, необходимо убедиться, что в последствии не потребуется информация о том, какие линейные объекты пересекают полигон. Если требуется, то сперва нужно определить именно это, а лишь затем производить операции над геометрией

Например, сперва нужно определить, какие дороги проходят через территорию, и лишь затем можно рассечь на полигоны в пределах кольца дорог. Это действие упрощает последующий сбор полевых данных.

После работы здесь можно перейти к обучению модели.

Ссылка на модель на основе логистической регрессии:
https://colab.research.google.com/drive/1iufWLzOG46Q8cdPvF1in1Ix8MEA9ltmX?usp=sharing

Ссылка на модель на основе случайного леса:

https://colab.research.google.com/drive/15Z6CWLqufYCQfDtDUmpNxE6vuP5Jv-mT?usp=sharing


#Расчёт доли длины границы с соседями определённого типа
Меняем

*   Ссылку на файл
*   Название поля, откуда берём названия типов
*   И по желанию можно изменить название выходного файла




In [ ]:
import geopandas as gpd

In [ ]:
# Загрузите полигональный слой
polygons = gpd.read_file('/content/updated_geodata_file.gpkg')

polygons = polygons.to_crs(epsg=4326)  # Эта строка меняет CRS (систему координат и проекцию). Если требуется, можно убрать

# Эта строка добавляет новый столбец. В одинарных ковычках название столюца. Его менять не надо
polygons['boundary_length'] = polygons.geometry.length

# Получаем уникальные значения в столюце, по которому считаем. Это тот столюец, который получали с помощью кода по ссулке
target_types = polygons['non-place'].unique()

# Перебор всех значений
for target_type in target_types:
    # Инициализация новых столбцов для длины границ соседей и доли общей границы
    neighbor_boundary_length_col = f'neighbor_boundary_length_{target_type}'
    neighbor_ratio_col = f'neighbor_ratio_{target_type}'

    polygons[neighbor_boundary_length_col] = 0.0 # исходные значения
    polygons[neighbor_ratio_col] = 0.0

    # Перебор объектов
    for idx, polygon in polygons.iterrows():
        # Выбор соседних полигонов одного типа
        neighbors = polygons[(polygons.geometry.touches(polygon.geometry)) & (polygons['non-place'] == target_type)]

        # Расчёт общей границы
        total_neighbor_length = neighbors['boundary_length'].sum()

        # Сохраняем длину границ  и считаем долю
        polygons.at[idx, neighbor_boundary_length_col] = total_neighbor_length
        if polygon['boundary_length'] > 0:
            polygons.at[idx, neighbor_ratio_col] = total_neighbor_length / polygon['boundary_length']

# Сохранение результата. Название файла можно менять. Формат файла должен совпадать со значением driver
polygons.to_file('updated_polygons.gpkg', layer='updated_layer', driver='GPKG')

print("Расчет доли длины границы завершен.")

<ipython-input-3-03426f86b8c1>:7: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons['boundary_length'] = polygons.geometry.length


Расчет доли длины границы завершен.


# Расстояние до ближайшей точки

Обязательно менять вназвании столбца

In [ ]:
import geopandas as gpd
from shapely.geometry import MultiPoint

# Пример загрузки GeoDataFrame
polygon_gdf = gpd.read_file('/content/output_b (1).gpkg') # полигоны
point_gdf = gpd.read_file('/content/subway.gpkg') # точки

# Убедитесь, что оба GeoDataFrame имеют одинаковую систему координат
polygon_gdf = polygon_gdf.to_crs(point_gdf.crs)

column_name = 'subway_distance'

def nearest_distance(polygon):
    centroid = polygon.geometry.centroid
    # Создаем MultiPoint из точек
    points = MultiPoint(point_gdf.geometry.tolist())

    # Вычисляем расстояния до всех точек
    distances = points.distance(centroid)

    # Если distances - это одно значение, обрабатываем его
    if isinstance(distances, float):
        return distances  # Возвращаем это значение, если оно одно
    else:
        distances_array = np.array(distances)
        return distances_array.min() if distances_array.size > 0 else float('inf')

# Применяем функцию к каждому полигону
polygon_gdf[column_name] = polygon_gdf.apply(nearest_distance, axis=1)

# Выводим результат
print(polygon_gdf[['geometry', column_name]])

# Сохранение результата в новый файл (если необходимо)
polygon_gdf.to_file(f'/content/updated_polygons_with_{column_name}.gpkg')  # путь для сохранения результата

                                               geometry  subway_distance
0     MULTIPOLYGON (((30.24991 59.87783, 30.24967 59...         0.004521
1     MULTIPOLYGON (((30.25671 59.8766, 30.2568 59.8...         0.003378
2     MULTIPOLYGON (((30.26618 59.87623, 30.26651 59...         0.006025
3     MULTIPOLYGON (((30.26606 59.87073, 30.26537 59...         0.005991
4     MULTIPOLYGON (((30.26606 59.87073, 30.26546 59...         0.006696
...                                                 ...              ...
6846  MULTIPOLYGON (((30.25503 59.87612, 30.25518 59...         0.005407
6847  MULTIPOLYGON (((30.25936 59.87607, 30.25943 59...         0.002118
6848  MULTIPOLYGON (((30.26089 59.87553, 30.26046 59...         0.002493
6849  MULTIPOLYGON (((30.26155 59.87526, 30.26188 59...         0.003108
6850  MULTIPOLYGON (((30.26377 59.87424, 30.26379 59...         0.005105

[6851 rows x 2 columns]


In [ ]:
polygon_gdf

,osm_id,osm_way_id,osm_version,osm_timestamp,osm_uid,osm_user,osm_changeset,other_tags,layer,path,...,shop_within_100m_kiosk,shop_within_10m_lottery,shop_within_50m_lottery,shop_within_100m_lottery,shop_within_10m_spices,shop_within_50m_spices,shop_within_100m_spices,tree_density,geometry,subway_distance
0,1411820,None,34.0,2023-09-16 23:21:35+00:00,8726248.0,traveleditor,141361719.0,"""landuse""=>""industrial"",""man_made""=>""works"",""n...",None,C:/Users/pc/Desktop/А.gpkg,...,3,4,4,4,1,1,1,7.046824e+06,"MULTIPOLYGON (((30.24991 59.87783, 30.24967 59...",0.004521
1,2069097,None,4.0,2019-02-14 09:41:59+00:00,392109.0,Константин Гам,67187748.0,"""area:highway""=>""footway"",""type""=>""multipolygon""",None,C:/Users/pc/Desktop/А.gpkg,...,3,4,4,4,1,1,1,0.000000e+00,"MULTIPOLYGON (((30.25671 59.8766, 30.2568 59.8...",0.003378
2,2587398,None,4.0,2022-11-13 19:13:38+00:00,9059129.0,ComradeSukhov,128859490.0,"""addr:housenumber""=>""6"",""addr:street""=>""улица ...",None,C:/Users/pc/Desktop/А.gpkg,...,3,4,4,4,1,1,1,0.000000e+00,"MULTIPOLYGON (((30.26618 59.87623, 30.26651 59...",0.006025
3,3333631,None,15.0,2024-05-22 04:48:46+00:00,6710042.0,dpolovinkin,151655525.0,"""amenity""=>""dentist"",""healthcare""=>""dentist"",""...",None,C:/Users/pc/Desktop/А.gpkg,...,3,4,4,4,1,1,1,0.000000e+00,"MULTIPOLYGON (((30.26606 59.87073, 30.26537 59...",0.005991
4,3333632,None,32.0,2024-02-07 04:29:38+00:00,3846961.0,mini-me,147159852.0,"""amenity""=>""kindergarten"",""name""=>""Детский сад...",None,C:/Users/pc/Desktop/А.gpkg,...,3,4,4,4,1,1,1,0.000000e+00,"MULTIPOLYGON (((30.26606 59.87073, 30.26546 59...",0.006696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6846,None,None,NaN,NaT,NaN,None,NaN,None,None,C:/Users/pc/Desktop/А.gpkg,...,3,4,4,4,1,1,1,0.000000e+00,"MULTIPOLYGON (((30.25503 59.87612, 30.25518 59...",0.005407
6847,None,None,NaN,NaT,NaN,None,NaN,None,None,C:/Users/pc/Desktop/А.gpkg,...,3,4,4,4,1,1,1,0.000000e+00,"MULTIPOLYGON (((30.25936 59.87607, 30.25943 59...",0.002118
6848,None,None,NaN,NaT,NaN,None,NaN,None,None,C:/Users/pc/Desktop/А.gpkg,...,3,4,4,4,1,1,1,0.000000e+00,"MULTIPOLYGON (((30.26089 59.87553, 30.26046 59...",0.002493
6849,None,None,NaN,NaT,NaN,None,NaN,None,None,C:/Users/pc/Desktop/А.gpkg,...,3,4,4,4,1,1,1,0.000000e+00,"MULTIPOLYGON (((30.26155 59.87526, 30.26188 59...",0.003108


# Подсчёт плотности точек

In [ ]:
import geopandas as gpd

In [ ]:
# Загрузите ваши данные
polygons_gdf = gpd.read_file('/content/predikt_0.84.gpkg')  # Полигоны
points_gdf = gpd.read_file('/content/gdf.tracks.gpkg')        # Точки
sawe_way = '/content/output_b.gpkg'

# Название столбца плотности

density_column='bench_density'

# Убедитесь, что оба GeoDataFrame имеют одинаковую систему координат
points_gdf = points_gdf.to_crs(polygons_gdf.crs)

# Подсчет количества точек в каждом полигоне
# Используем метод sjoin для пространственного соединения
joined = gpd.sjoin(points_gdf, polygons_gdf, how='inner', predicate='within')

# Подсчет количества точек в каждом полигоне
count_per_polygon = joined.groupby('index_right').size()

# Присоединяем подсчеты к полигональному геодатафрейму
polygons_gdf['point_count'] = polygons_gdf.index.map(count_per_polygon).fillna(0)

# Вычисление площади полигонов
polygons_gdf['area'] = polygons_gdf.geometry.area

# Функция для вычисления плотности точек
def calculate_density(points_column='point_count', area_column='area', density_column='density'):
    polygons_gdf[density_column] = polygons_gdf[points_column] / polygons_gdf[area_column]
    return polygons_gdf

# Вызываем функцию для расчета плотности. density_column='bench_density' - здесь нужно изменить название в ковычках так, чтобы потом вы узнали этот столбец
# Он будет использован при обучении в неизменном виде
polygons_gdf = calculate_density(points_column='point_count', area_column='area', density_column=density_column)

# Сохранение результата в новый файл (если нужно)
polygons_gdf.to_file(sawe_way)  # Замените на ваш путь

In [ ]:
polygons_gdf

,non-place1,I.predikt,II.predikt,III.predikt,non-place.predikt,pustoe.predikt,geometry,point_count,area,bench_density
0,II,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((3367404.785 8372587.3, 3367377...",481.0,711414.336212,0.000676
1,None,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((3368161.695 8372314.825, 33681...",259.0,13833.033284,0.018723
2,II,0.000025,0.999446,0.000113,0.000346,0.000069,"MULTIPOLYGON (((3369215.879 8372233.087, 33692...",0.0,2478.569369,0.000000
3,II,0.000035,0.999682,0.000016,0.000154,0.000114,"MULTIPOLYGON (((3369202.454 8371012.592, 33691...",0.0,8008.417196,0.000000
4,II,0.000025,0.999446,0.000113,0.000346,0.000069,"MULTIPOLYGON (((3369202.454 8371012.592, 33691...",0.0,8825.514149,0.000000
...,...,...,...,...,...,...,...,...,...,...
6822,III,0.334475,0.182379,0.214973,0.212254,0.055918,"MULTIPOLYGON (((3367974.331 8372207.807, 33679...",0.0,26446.335731,0.000000
6823,III,0.334475,0.182379,0.214973,0.212254,0.055918,"MULTIPOLYGON (((3368456.312 8372198.142, 33684...",0.0,4326.070139,0.000000
6824,III,0.334475,0.182379,0.214973,0.212254,0.055918,"MULTIPOLYGON (((3368626.932 8372076.923, 33685...",0.0,2942.985749,0.000000
6825,III,0.334475,0.182379,0.214973,0.212254,0.055918,"MULTIPOLYGON (((3368699.996 8372017.143, 33687...",0.0,5116.662581,0.000000


# Добавление данных о классах дорог

In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
# Загрузка слоев
roads = gpd.read_file('/content/дороги.gpkg')  # путь к слою с дорогами
polygons = gpd.read_file('/content/output_b.gpkg')  # путь к слою с полигонами

# Надо проверить, что оба слоя имеют одинаковую систему координат. Если есть различия, то всё уладит метод .to_crs. Он там есть, переживать и менять ничего не надо
if roads.crs != polygons.crs:
    roads = roads.to_crs(polygons.crs)

# Функция для получения классов дорог, пересекающих полигон
def get_road_classes(polygon):
    intersecting_roads = roads[roads.intersects(polygon)]
    # Собираем классы дорог, фильтруем None и объединяем их в строку через запятую
    road_classes = intersecting_roads['highway'].dropna().unique()  # Удаляем None значения
    return ', '.join(road_classes)  # Теперь здесь не будет None

# Применяем функцию к каждому полигону и создаем новый столбец 'road'
polygons['road'] = polygons.geometry.apply(get_road_classes)

# Сохранение результата в новый файл (если необходимо)
polygons.to_file('/content/roads_in_polygon.gpkg')  # путь для сохранения результата

In [ ]:
roads

,full_id,osm_id,osm_type,highway,tracktype,parking:both:restriction,survey:date,informal,was:highway,maxspeed:type,...,name:etymology:wikidata,name,surface,smoothness,oneway,maxspeed,lit,lanes,foot,geometry
0,w23873900,23873900,way,unclassified,None,None,None,None,None,None,...,Q155863,Кронштадтская улица,asphalt,None,yes,RU:urban,no,2,no,"MULTILINESTRING ((30.25853 59.878, 30.25827 59..."
1,w31370823,31370823,way,primary,None,None,None,None,None,None,...,None,проспект Стачек,asphalt,very_good,yes,RU:urban,yes,3,no,"MULTILINESTRING ((30.26144 59.88033, 30.26072 ..."
2,w31386011,31386011,way,primary,None,None,None,None,None,None,...,Q166073,Комсомольская площадь,asphalt,very_good,yes,RU:urban,yes,4,no,"MULTILINESTRING ((30.25768 59.87693, 30.25751 ..."
3,w31400771,31400771,way,primary,None,None,None,None,None,None,...,Q166073,Комсомольская площадь,asphalt,very_good,yes,RU:urban,yes,2,no,"MULTILINESTRING ((30.25846 59.87668, 30.25837 ..."
4,w31400780,31400780,way,residential,None,None,None,None,None,None,...,Q4692,улица Возрождения,asphalt,excellent,None,RU:urban,yes,2,None,"MULTILINESTRING ((30.2703 59.88199, 30.27011 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12807,w1274970857,1274970857,way,footway,None,None,None,None,None,None,...,None,None,asphalt,None,None,None,None,None,None,"MULTILINESTRING ((30.4145 59.89143, 30.41451 5..."
12808,w1274970860,1274970860,way,footway,None,None,None,None,None,None,...,None,None,asphalt,None,None,None,None,None,None,"MULTILINESTRING ((30.4122 59.8893, 30.41211 59..."
12809,w1285998453,1285998453,way,path,None,None,None,None,None,None,...,None,None,ground,None,None,None,None,None,None,"MULTILINESTRING ((30.42162 59.90104, 30.42157 ..."
12810,w1316324521,1316324521,way,proposed,None,None,None,None,None,None,...,None,None,None,None,yes,RU:urban,yes,None,None,"MULTILINESTRING ((30.42423 59.90261, 30.42415 ..."


Следуюая ячейка необязательна, т.к. этот же блок есть в блокноте с обучениме моделей

В этой ячейке мы получаем более сложную для восприятия, но простую для обучения таблицу: теперь будет не просто столбец, в котором есть данные о всех дорогах, пересекающих полигон, а столцы с вариантами дорог, поторые могут пересекаться с полигонм и значениями 1 и 0 (есть пересечение или его нет)

In [ ]:
# Укажите название столбца, который нужно обработать
column_name = 'road'

# Получаем уникальные значения из указанного столбца
unique_values = polygons[column_name].unique()

# Фильтруем уникальные значения, которые содержат запятые
values_with_commas = [value for value in unique_values if isinstance(value, str) and ',' in value]

# Создаем новый список уникальных значений, разделяя по запятой
new_unique_values = set()
for value in values_with_commas:
    split_values = value.split(',')
    new_unique_values.update([v.strip() for v in split_values])

# Создаем новые столбцы для каждого уникального значения
for unique_value in new_unique_values:
    new_column_name = f"{column_name}={unique_value}"
    polygons[new_column_name] = polygons[column_name].apply(lambda x: 1 if unique_value in str(x) else 0)

# Теперь gdf содержит новые столбцы с бинарными значениями
print(polygons.head())

# Сохранение результата в новый файл (если необходимо)
polygons.to_file('/content/roadss.gpkg')

                      name          type       amenity admin_leve barrier  \
0            ?????????????          None          None       None    None   
1  ????????????? ?????????  multipolygon          None       None    None   
2  ????????????? ?????????  multipolygon          None       None    None   
3                     None          None          None       None    None   
4         ??????? ??? ? 36          None  kindergarten       None    None   

  boundary      building historic     landuse leisure  ... road=service  \
0     None        office     None        None    None  ...            0   
1     None          None     None  industrial    None  ...            1   
2     None          None     None  industrial    None  ...            0   
3     None          None     None        None    None  ...            1   
4     None  kindergarten     None        None    None  ...            0   

  road=construction road=corridor road=primary_link road=cycleway  \
0                

# Счёт точек на расстоянии
Например, магазинов

In [ ]:
import geopandas as gpd
import pandas as pd

# Загрузка полигональных данных в GeoDataFrame
polygons_gdf = gpd.read_file('/content/ПОЛИГОНЫ.gpkg')  # Полигоны
# Загрузка точечных данных в GeoDataFrame
points_gdf = gpd.read_file('/content/food.gpkg')  # Точки
# Поле классификации объектов
fild = points_gdf['amenity']

# Убедитесь, что CRS (координатная система) совпадают
if polygons_gdf.crs != points_gdf.crs:
    points_gdf = points_gdf.to_crs(polygons_gdf.crs)

# Получаем уникальные значения из столбца 'shop'
unique_shops = fild.unique()

# Создаем новые столбцы для каждого уникального значения в 'shop'
for shop in unique_shops:
    # Фильтруем точки по текущему значению 'shop'
    filtered_points = points_gdf[fild == shop]

    # Функция для подсчета точек в буферах
    def count_points_in_buffer(buffer_distance):
        # Создаем буфер вокруг полигонов
        buffered_polygons = polygons_gdf.geometry.buffer(buffer_distance)
        # Считаем количество точек в каждом буфере
        count = buffered_polygons.apply(lambda polygon: filtered_points.geometry.within(polygon).sum())
        return count

    # Добавляем новые столбцы с количеством точек на расстоянии 100, 500 и 1000 м
    polygons_gdf[f'food_within_10m_{shop}'] = count_points_in_buffer(10)
    polygons_gdf[f'food_within_50m_{shop}'] = count_points_in_buffer(50)
    polygons_gdf[f'food_within_100m_{shop}'] = count_points_in_buffer(100)

# Сохраняем результат в новый файл
polygons_gdf.to_file('output_with_point_counts.gpkg', driver='GPKG')